In [1]:
import opensim as osim
import pandas as pd
import numpy as np
import os

In [2]:
def extract_marker_locations(model_path):
    # Load the model
    model = osim.Model(model_path)
    
    # Initialize the system and get the default state
    state = model.initSystem()
    
    # Get the marker set from the model
    marker_set = model.getMarkerSet()
    
    # Create a dictionary to store marker locations
    marker_locations = {}
    
    # Loop through all markers and get their locations in ground frame
    for i in range(marker_set.getSize()):
        marker = marker_set.get(i)
        
        # Get the marker's location in ground frame
        location = marker.getLocationInGround(state)
        
        # Convert SimTK::Vec3 to a regular Python list or numpy array if needed
        location_array = np.array([location.get(0), location.get(1), location.get(2)])
        
        # Store in dictionary with marker name as key
        marker_locations[marker.getName()] = location_array
    
    return marker_locations


def data_frame(dict) -> pd.DataFrame:
    mrkrs_new = []
    for mrkr, location in dict.items():
        dic = {}
        dic['name'] = mrkr
        dic['r'] = location[0]*1000
        dic['a'] = location[1]*1000
        dic['s'] = location[2]*1000
        mrkrs_new.append(dic)

    # create the dataframe that contains all information
    osim_bone_markers_df = pd.DataFrame.from_dict(mrkrs_new)
    osim_bone_markers_df.set_index('name', inplace=True)
    return osim_bone_markers_df

In [3]:
# load the generic model with bony markers from the template folder
generic_model_with_bone_landmarks = os.path.join(f'../../', 'templates','RajagopalModified_generic_copy_with_bone_markers.osim')

# load the generic model without the bony markers from the template folder of the individual
generic_model = os.path.join(f'../../', 'templates','RajagopalModified_generic_copy.osim')



In [4]:
# load landmarks in ground frame for each model
markers_with_bone_in_ground = extract_marker_locations(generic_model_with_bone_landmarks)
markers_skin_only_in_ground = extract_marker_locations(generic_model)

# create a list of bone landmarks
bone_landmark_names = [name for name in markers_with_bone_in_ground.keys() if name not in markers_skin_only_in_ground.keys()]

In [ ]:
# dictinary of the landmarks of interest
bone_markers_in_ground = {name: markers_with_bone_in_ground[name] for name in bone_landmark_names}

In [6]:
bone_markers_in_ground_df = data_frame(bone_markers_in_ground)
bone_markers_in_ground_df.to_csv(os.path.join(f'../../', 'templates', 'bone_markers_in_ground.csv'), index=True)